In [96]:
from qiskit import *
from qiskit.circuit.library import *
from qiskit.primitives import Sampler
import numpy as np 
from qiskit.extensions import HamiltonianGate

A = [
        [1,-1/3],
        [-1/3,1]
    ]

# A = [
#         [0,1],
#         [1,0]
#     ]

precision = 2
qreg = QuantumRegister(precision)
qregb = QuantumRegister(1)
creg = ClassicalRegister(precision)

QC = QuantumCircuit(qreg, qregb, creg)
QC.x(-1)
pe = PhaseEstimation(precision, HamiltonianGate(A, 3*np.pi/4), QFT(precision, inverse=True, do_swaps=True))

QC.append(pe, range(precision+1))
QC.measure(qreg, creg)
QC.decompose().decompose().decompose().draw()


┌───┐                                      ┌──────────┐┌─┐   
q2787_0: ───┤ H ├────────────■─────────────────────────┤0         ├┤M├───
            ├───┤            │                         │  IQFT_dg │└╥┘┌─┐
q2787_1: ───┤ H ├────────────┼────────────────■────────┤1         ├─╫─┤M├
         ┌──┴───┴───┐┌───────┴───────┐┌───────┴───────┐└──────────┘ ║ └╥┘
  q2788: ┤ U(π,0,π) ├┤ hamiltonian^1 ├┤ hamiltonian^2 ├─────────────╫──╫─
         └──────────┘└───────────────┘└───────────────┘             ║  ║ 
  c49: 2/═══════════════════════════════════════════════════════════╩══╩═
                                                                    0  1

In [97]:
def binaryToPhase(binary):
    binary=list(binary[::-1])
    length = len(binary)
    res=0
    
    for i in range(length):
        digit=int(binary.pop())
        if digit==1:
            res=res+pow(2,i)  
    return res

job = Sampler().run(QC, shots=100)
for k,v in job.result().quasi_dists[0].items():
    print (bin(k)[2:], v, binaryToPhase(bin(k)[2:]))

10 0.47 1
11 0.53 3
